In [49]:
import pandas as pd
import opendatasets as od

In [50]:
# od.download("https://www.kaggle.com/competitions/spaceship-titanic")

In [51]:
raw=pd.read_csv('spaceship-titanic/train.csv')
# Xtrain.head()

In [52]:
ytrain=raw['Transported']
Xtrain=raw.drop(columns=['Transported','Name'])
Xtrain=Xtrain.assign(
    RoomService=Xtrain.RoomService.fillna(0).replace('nan',0),
    CyroSleep=Xtrain.CryoSleep.fillna(False).replace('nan',0),
    VIP=Xtrain.VIP.fillna(False).replace('nan',0),
    FoodCourt=Xtrain.FoodCourt.fillna(0).replace('nan',0),
    ShoppingMall=Xtrain.ShoppingMall.fillna(0).replace('nan',0),
    Spa=Xtrain.Spa.fillna(0).replace('nan',0),
    VRDeck=Xtrain.VRDeck.fillna(0).replace('nan',0),

)


In [57]:
Xtrain.HomePlanet.unique()

array(['Europa', 'Earth', 'Mars', nan], dtype=object)

In [58]:
Xtrain.Destination.unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e', nan], dtype=object)

In [53]:
(Xtrain
 .assign(Transported=ytrain==True)
 .corr(method='spearman')
 .style
 .background_gradient(cmap='RdBu',vmax=1,vmin=-1)
 .set_sticky(axis='index')
)

,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,CyroSleep,Transported
Age,1.000000,0.102795,0.123346,0.203007,0.101755,0.193787,0.179270,-0.074679,-0.071029
VIP,0.102795,1.000000,0.051972,0.111360,0.033293,0.086885,0.094900,-0.078281,-0.037261
RoomService,0.123346,0.051972,1.000000,0.181157,0.430106,0.239521,0.180872,-0.508773,-0.363899
FoodCourt,0.203007,0.111360,0.181157,1.000000,0.185407,0.470270,0.497026,-0.522937,-0.186388
ShoppingMall,0.101755,0.033293,0.430106,0.185407,1.000000,0.253359,0.187538,-0.504369,-0.227018
Spa,0.193787,0.086885,0.239521,0.470270,0.253359,1.000000,0.437234,-0.538649,-0.366099
VRDeck,0.179270,0.094900,0.180872,0.497026,0.187538,0.437234,1.000000,-0.517701,-0.347008
CyroSleep,-0.074679,-0.078281,-0.508773,-0.522937,-0.504369,-0.538649,-0.517701,1.000000,0.460132
Transported,-0.071029,-0.037261,-0.363899,-0.186388,-0.227018,-0.366099,-0.347008,0.460132,1.000000


It seems that the persons who have billed luxury amenities have a negative relationship with being transported. In simple words, the people who have bills are less likely to be transported.

In [54]:
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn import tree

stump_dt=tree.DecisionTreeClassifier(max_depth=1)
stump_dt.fit(Xtrain,ytrain)

ValueError: could not convert string to float: 'Europa'

In [ ]:
fig,ax=plt.subplots(figsize=(8,4))
features= list(c for c in Xtrain.columns)
tree.plot_tree(stump_dt,
               feature_names=features,
               filled=True,
               class_names=stump_dt.classes_,
               ax=ax)